# Azure Vision Implementaion - Dima 

## Notebook Overview

On a high level this notebook utilizes Azure AI Document Intelligence Studio to extract text from a set of Herbarium specimens, extracted text is then fed into OpenAI's GPT-4 Turbo model to extract relevant information in DARWIN JSON format.

The notebook has several different components: 
1. A resizing function that takes an input of a folder of images and resizes all those images to a size smaller than 4mb-
    Azure Vision AI requires input images to be 4mb or smaller, run the resizing function as needed.
2. Making a UI- running this code creates a friendly UI in the notebook itself. User is able to upload a Herbarium specimen image, click submit, and in 20 seconds obtain extracted results in DARWIN JSON format.
3. Saving Results as a .txt- this code takes an input of a folder of Herbarium images, runs them through Azure Vision OCR to extract all text, then all the extracted text is ran through GPT-4 Turbo to extract just the content that is DARWIN JSON relevant format. Each result is then saved in an output folder in .txt format. 
4. Running model on Cyrillic and Chinese images- same as the previous code however tailored to Cyrillic and Chinese images through custom prompting. Results saved as a .txt file as well.
5. Saving Results as a PDF- same as the previous code that saves as a result as a .txt, however offers more insights. By saving as a pdf you are able to view the original image, the image as processed by Azure AI and how Azure AI identifies sections with text, and all text identified by Azure with confidence for each term all in one pdf file. Note: The text is processed by GPT-4 Turbo however that output is not saved in the pdf, you can however see that output in this notebook as an example. 
6. Experimenting with Cropping Images- This code was created to see if results code be improved by cropping out all parts of an input image except the sections with identified text. Current results produced by the cropping process are worse compared to not cropping, indicating how the current cropping function is too aggressive and needs to be further worked on. 


Note: Code section 2-6 all require an Azure and OpenAI API key. 


In [4]:
#!pip install azure-ai-formrecognizer --pre
#!pip install opencv-python-headless matplotlib
#!pip install matplotlib pillow
#!pip install ipywidgets
#!pip install shapely
#!pip install openai
#!pip install reportlab
#!pip install --upgrade gradio

In [4]:
# Standard library imports
import os
import re
import io

# External libraries for image processing and visualization
from PIL import Image, ImageDraw, ImageFont
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

# Azure AI services
from azure.core.credentials import AzureKeyCredential
from azure.ai.formrecognizer import DocumentAnalysisClient

# PDF generation library
from reportlab.lib.pagesizes import letter
from reportlab.pdfgen import canvas
from reportlab.lib.units import inch

# Gradio for creating interfaces
import gradio as gr

# Other libraries
import numpy as np
import openai

# 1. Resizing images to smaller size for API to accept them

In [21]:
def resize_image(input_path, output_path, max_size_mb, quality=85):
    """
    Resize the image found at input_path and save it to output_path.
    The image is resized to be under max_size_mb megabytes.
    """
    # Load the image
    with Image.open(input_path) as img:
        # Calculate target size to maintain aspect ratio
        ratio = img.width / img.height
        target_width = int((max_size_mb * 1024 * 1024 * ratio) ** 0.5)
        target_height = int(target_width / ratio)

        # Resize the image
        resized_img = img.resize((target_width, target_height), Image.Resampling.LANCZOS)

        # Save the resized image
        resized_img.save(output_path, quality=quality)

# change input/ output folders as needed        
input_folder = '/projectnb/sparkgrp/ml-herbarium-grp/ml-herbarium-data/TROCR_Training/goodfiles/'
output_folder = '/projectnb/sparkgrp/ml-herbarium-grp/fall2023/resized-images'

if not os.path.exists(output_folder):
    os.makedirs(output_folder)

for file_name in os.listdir(input_folder):
    file_path = os.path.join(input_folder, file_name)
    output_file_path = os.path.join(output_folder, file_name)

    # Check if the file is an image
    if file_path.lower().endswith(('.png', '.jpg', '.jpeg')):
        resize_image(file_path, output_file_path, max_size_mb=4)  


# 2. Making a UI

In [6]:
# Azure Cognitive Services endpoint and key
endpoint = "https://herbariumsamplerecognition.cognitiveservices.azure.com/"
key = "your key here"


def sanitize_filename(filename):
    # Remove characters that are not alphanumeric, spaces, dots, or underscores
    return re.sub(r'[^\w\s\.-]', '', filename)

def extract_info(text):
    # Set your OpenAI API key
    openai.api_key = 'your key here'

    # Prepare the prompt for the API
    prompt = f"From the provided text, return only the relevant information in a JSON format according to the Darwin Core standard for biodiversity specimen. Note: make sure that each output has a 'country' field. If you do not find an explicit country, make your best guess at the country using the context of the other text.\n{text}\n{text}"

    try:
        # Send the request to the API
        response = openai.ChatCompletion.create(
            model="gpt-4-1106-preview",
            messages=[{"role": "system", "content": "You are a helpful assistant."}, 
                      {"role": "user", "content": prompt}]
        )

        # Extract the response
        return response.choices[0].message['content'] if response.choices else "No response from the API."

    except Exception as e:
        return f"An error occurred: {str(e)}"

def analyze_read(image_stream):
    try:
        document_analysis_client = DocumentAnalysisClient(
            endpoint=endpoint, credential=AzureKeyCredential(key)
        )

        poller = document_analysis_client.begin_analyze_document(
            "prebuilt-read", image_stream)
        result = poller.result()

        # Collect the content from the document
        document_content = result.content
        extracted_info = extract_info(document_content)

        return extracted_info

    except Exception as e:
        return f"An error occurred: {str(e)}"

def model_function(image):
    # Convert the NumPy array to a PIL Image object
    image = Image.fromarray(np.uint8(image)).convert('RGB')

    # Convert the uploaded image to a byte stream
    image_bytes = io.BytesIO()
    image.save(image_bytes, format='JPEG')  # Using 'JPEG' as the format
    image_bytes = image_bytes.getvalue()

    output_text = analyze_read(image_bytes)
    return output_text

iface = gr.Interface(fn=model_function, inputs="image", outputs="text")
iface.launch(share=True)


Running on local URL:  http://127.0.0.1:7862
Running on public URL: https://5b5e36d606e49cc031.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


# 3. Saving Results as a .txt 

In [ ]:
# Azure Cognitive Services endpoint and key
endpoint = "https://herbariumsamplerecognition.cognitiveservices.azure.com/"
key = "your key here"

def sanitize_filename(filename):
    # Remove characters that are not alphanumeric, spaces, dots, or underscores
    return re.sub(r'[^\w\s\.-]', '', filename)

def extract_info(text):
    # Set your OpenAI API key
    openai.api_key = 'your key here'

    # Prepare the prompt for the API
    prompt = f"From the provided text, return only the relevant information in a JSON format according to the Darwin Core standard for biodiversity specimen. Note: make sure that each output has a 'country' field. If you do not find an explicit country, make your best guess at the country using the context of the other text.\n{text}\n{text}"

    try:
        # Send the request to the API
        response = openai.ChatCompletion.create(
            model="gpt-4-1106-preview",
            messages=[{"role": "system", "content": "You are a helpful assistant."}, 
                      {"role": "user", "content": prompt}]
        )

        # Extract the response
        return response.choices[0].message['content'] if response.choices else "No response from the API."

    except Exception as e:
        return f"An error occurred: {str(e)}"

def analyze_read(image_path, output_path):
    try:
        with open(image_path, "rb") as f:
            image_stream = f.read()

        document_analysis_client = DocumentAnalysisClient(
            endpoint=endpoint, credential=AzureKeyCredential(key)
        )

        poller = document_analysis_client.begin_analyze_document(
            "prebuilt-read", image_stream)
        result = poller.result()

        # Collect the content from the document
        document_content = result.content
        extracted_info = extract_info(document_content)

        # Save the extracted information to a text file
        output_filename = os.path.join(output_path, sanitize_filename(os.path.basename(image_path).replace('.jpg', '.txt')))
        with open(output_filename, 'w') as text_file:
            text_file.write(extracted_info)

    except Exception as e:
        print(f"An error occurred while processing {image_path}: {e}")

if __name__ == "__main__":
    input_folder = '/projectnb/sparkgrp/ml-herbarium-grp/fall2023/resized-images'
    output_folder = '/projectnb/sparkgrp/ml-herbarium-grp/fall2023/AzureVisionResults'

    # Create the output folder if it doesn't exist
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    image_counter = 0  # Initialize a counter for the number of images processed

    # Iterate over each image in the input folder
    for image_file in os.listdir(input_folder):
        image_path = os.path.join(input_folder, image_file)
        
        # Check if the file is an image
        if image_path.lower().endswith(('.png', '.jpg', '.jpeg')):
            analyze_read(image_path, output_folder)
            image_counter += 1  # Increment the counter

            if image_counter >= 224:  # Stop after processing x images
                break


# 4. Running Model On Cyrillic & Chinese Images

In [1]:
# Azure Cognitive Services endpoint and key
endpoint = "https://herbariumsamplerecognition.cognitiveservices.azure.com/"
key = "your key here"

def sanitize_filename(filename):
    # Remove characters that are not alphanumeric, spaces, dots, or underscores
    return re.sub(r'[^\w\s\.-]', '', filename)

def extract_info(text):
    # Set your OpenAI API key
    openai.api_key = 'your key here'

    # Prompt for cyrillic images
    prompt = f"From the provided text, return only the relevant information in a JSON format according to the Darwin Core standard for biodiversity specimen. NOTE: Parts or the majority of the textmay be in Cyrillic. Take this into consideration. Additionally, there should be a 'country' field, if you cannot determinethe country directly, to your best ability, infer what the country is:\n{text}"
    
    # Prompt for Chinese images
    #prompt = f"From the provided text, return only the relevant information in a JSON format according to the Darwin Core standard for biodiversity specimen. NOTE: Parts or the majority of the textmay be in Chinese, Take this into consideration. Additionally, there should be a 'country' field, if you cannot determinethe country directly, to your best ability, infer what the country is. Lastly, the more info the better, output chinese character as appropriate:\n{text}"

    try:
        # Send the request to the API
        response = openai.ChatCompletion.create(
            model="gpt-4-1106-preview",
            messages=[{"role": "system", "content": "You are a helpful assistant."}, 
                      {"role": "user", "content": prompt}]
        )

        # Extract the response
        return response.choices[0].message['content'] if response.choices else "No response from the API."

    except Exception as e:
        return f"An error occurred: {str(e)}"

def analyze_read(image_path, output_path):
    try:
        with open(image_path, "rb") as f:
            image_stream = f.read()

        document_analysis_client = DocumentAnalysisClient(
            endpoint=endpoint, credential=AzureKeyCredential(key)
        )

        poller = document_analysis_client.begin_analyze_document(
            "prebuilt-read", image_stream)
        result = poller.result()

        # Collect the content from the document
        document_content = result.content
        extracted_info = extract_info(document_content)
        
        #print(extracted_info)


        # Save the extracted information to a text file
        output_filename = os.path.join(output_path, sanitize_filename(os.path.basename(image_path).replace('.jpg', '.txt')))
        with open(output_filename, 'w') as text_file:
            text_file.write(extracted_info)

    except Exception as e:
        print(f"An error occurred while processing {image_path}: {e}")

if __name__ == "__main__":
    #input_folder = '/projectnb/sparkgrp/ml-herbarium-grp/fall2023/CyrillicImages'
    #output_folder = '/projectnb/sparkgrp/ml-herbarium-grp/fall2023/CyrillicResults'
    input_folder = '/projectnb/sparkgrp/ml-herbarium-grp/fall2023/ChineseImages'
    output_folder = '/projectnb/sparkgrp/ml-herbarium-grp/fall2023/ChineseResults'

    # Create the output folder if it doesn't exist
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    image_counter = 0  # Initialize a counter for the number of images processed

    # Iterate over each image in the input folder
    for image_file in os.listdir(input_folder):
        image_path = os.path.join(input_folder, image_file)
        
        # Check if the file is an image
        if image_path.lower().endswith(('.png', '.jpg', '.jpeg')):
            analyze_read(image_path, output_folder)
            image_counter += 1  # Increment the counter

            if image_counter >= 35:  # Stop after processing x images
                break


# 5. Saving Results as a PDF

In [3]:
# Azure Cognitive Services endpoint and key
endpoint = "https://herbariumsamplerecognition.cognitiveservices.azure.com/"
key = "your key here"

def sanitize_filename(filename):
    # Remove characters that are not alphanumeric, spaces, dots, or underscores
    return re.sub(r'[^\w\s\.-]', '', filename)

def format_bounding_box(bounding_box):
    if not bounding_box:
        return "N/A"
    return ", ".join(["[{}, {}]".format(p.x, p.y) for p in bounding_box])


def draw_boxes(image_path, words):
    original_image = Image.open(image_path)
    annotated_image = original_image.copy()
    draw = ImageDraw.Draw(annotated_image)

    for word in words:
        polygon = word['polygon']
        if polygon:
            bbox = [(point.x, point.y) for point in polygon]
            try:
                # Replace special characters that cannot be encoded in 'latin-1'
                text_content = word['content'].encode('ascii', 'ignore').decode('ascii')
            except Exception as e:
                print(f"Error processing text {word['content']}: {e}")
                text_content = "Error"
            draw.polygon(bbox, outline="red")
            draw.text((bbox[0][0], bbox[0][1]), text_content, fill="green")

    
    return annotated_image

page_width, page_height = letter 

# Function to calculate scale to fit the image within page dimensions
def calculate_scale(image, max_width, max_height):
    scale_w = max_width / image.width
    scale_h = max_height / image.height
    return min(scale_w, scale_h)


def extract_info(text):
    # Set your OpenAI API key
    openai.api_key = 'your key here'

    # Prepare the prompt for the API
    prompt = f"From the provided text, return only the relevant information in a JSON format according to the Darwin Core standard for biodiversity specimen:\n{text}"

    try:
        # Send the request to the API
        response = openai.ChatCompletion.create(
            model="gpt-4-1106-preview",
            messages=[{"role": "system", "content": "You are a helpful assistant."}, 
                      {"role": "user", "content": prompt}]
        )

        # Extract the response
        if response.choices:
            return response.choices[0].message['content']
        else:
            return "No response from the API."

    except Exception as e:
        return f"An error occurred: {str(e)}"


def analyze_read(image_path, output_path):
    try:
        with open(image_path, "rb") as f:
            image_stream = f.read()

        document_analysis_client = DocumentAnalysisClient(
            endpoint=endpoint, credential=AzureKeyCredential(key)
        )

        poller = document_analysis_client.begin_analyze_document(
            "prebuilt-read", image_stream)
        result = poller.result()

       # Collect words, their polygon data, and confidence
        words = []
        confidence_text = ""
        for page in result.pages:
            for word in page.words:
                words.append({
                    'content': word.content,
                    'polygon': word.polygon
                })
                confidence_text += "'{}' confidence {}\n".format(word.content, word.confidence)

        document_content = result.content + "\n\nConfidence Metrics:\n" + confidence_text
        extracted_info = extract_info(document_content)
        print(extracted_info)
        original_image = Image.open(image_path)
        annotated_img = draw_boxes(image_path, words)
        

        # Set up PDF
        output_filename = os.path.join(output_path, sanitize_filename(os.path.basename(image_path).replace('.jpg', '.pdf')))
        c = canvas.Canvas(output_filename, pagesize=letter)
        width, height = letter  # usually 612 x 792

        # Draw original image
        if original_image.height <= height:
            c.drawImage(image_path, 0, height - original_image.height, width=original_image.width, height=original_image.height, mask='auto')
            y_position = height - original_image.height
        else:
            # Handle large images or add scaling logic here
            pass

        
        # Draw original image
        scale = calculate_scale(original_image, page_width, page_height)
        img_width, img_height = original_image.width * scale, original_image.height * scale
        c.drawImage(image_path, 0, page_height - img_height, width=img_width, height=img_height, mask='auto')
        y_position = page_height - img_height

        # Draw annotated image
        annotated_image_path = '/tmp/annotated_image.png'
        annotated_img.save(annotated_image_path)
        scale = calculate_scale(annotated_img, page_width, page_height)
        annotated_img_width, annotated_img_height = annotated_img.width * scale, annotated_img.height * scale
        if y_position - annotated_img_height >= 0:
            c.drawImage(annotated_image_path, 0, y_position - annotated_img_height, width=annotated_img_width, height=annotated_img_height, mask='auto')
        else:
            c.showPage()
            c.drawImage(annotated_image_path, 0, page_height - annotated_img_height, width=annotated_img_width, height=annotated_img_height, mask='auto')
        

        # Add text
        textobject = c.beginText()
        textobject.setTextOrigin(10, y_position - 15)
        textobject.setFont("Times-Roman", 12)

        for line in document_content.split('\n'):
            if textobject.getY() - 15 < 0:  # Check if new page is needed for more text
                c.drawText(textobject)
                c.showPage()
                textobject = c.beginText()
                textobject.setTextOrigin(10, height - 15)
                textobject.setFont("Times-Roman", 12)
            textobject.textLine(line)
        
        c.drawText(textobject)
        c.save()

    except Exception as e:
        print(f"An error occurred while processing {image_path}: {e}")


if __name__ == "__main__":
    input_folder = '/projectnb/sparkgrp/ml-herbarium-grp/fall2023/resized-images'
    output_folder = '/projectnb/sparkgrp/ml-herbarium-grp/fall2023/temp'

    # Create the output folder if it doesn't exist
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    image_counter = 0  # Initialize a counter for the number of images processed

    # Iterate over each image in the input folder
    for image_file in os.listdir(input_folder):
        image_path = os.path.join(input_folder, image_file)
        
        # Check if the file is an image
        if image_path.lower().endswith(('.png', '.jpg', '.jpeg')):
            analyze_read(image_path, output_folder)
            image_counter += 1  # Increment the counter

            if image_counter >= 5:  # Stop after processing x images
                break





```json
{
  "occurrenceID": "00592128",
  "scientificName": "Atriplex patula L. var. hastata Gray",
  "recordedBy": "Kate Furbish",
  "country": "USA",
  "stateProvince": "Maine",
  "county": "Washington",
  "locality": "North Lubec",
  "eventDate": "1902-09-13",
  "institutionCode": "Harvard University Herbaria",
  "catalogNumber": "00592128",
  "basisOfRecord": "PreservedSpecimen",
  "rightsHolder": "President and Fellows of Harvard College"
}
```
Below is the JSON representation of the relevant Darwin Core fields extracted from the provided text. The Darwin Core standard has many possible fields, but I've put together the ones that can be inferred from the text you've supplied:

```json
{
  "type": "SYNTYPE",
  "scientificName": "Glaux maritima var. obtusifolia Fernald",
  "acceptedNameUsage": "Glaux maritima L.",
  "verbatimEventDate": "July 24, 1902",
  "recordNumber": "00936578, 00936579",
  "locality": "Brackish marsh, St. Andrews, BATHURST, GLOUCESTER COUNTY",
  "stateProvince"

# 6. Experimenting With Cropping Images

So far results have been better for images that have not been cropped (cropping function needs improvement)

In [58]:
# Azure Cognitive Services endpoint and key
endpoint = "https://herbariumsamplerecognition.cognitiveservices.azure.com/"
key = "AZURE KEY HERE"

def sanitize_filename(filename):
    # Remove characters that are not alphanumeric, spaces, dots, or underscores
    return re.sub(r'[^\w\s\.-]', '', filename)

def format_bounding_box(bounding_box):
    if not bounding_box:
        return "N/A"
    return ", ".join(["[{}, {}]".format(p.x, p.y) for p in bounding_box])

def draw_boxes(image_path, words):
    original_image = Image.open(image_path)
    annotated_image = original_image.copy()
    draw = ImageDraw.Draw(annotated_image)

    for word in words:
        polygon = word['polygon']
        if polygon:
            bbox = [(point.x, point.y) for point in polygon]
            try:
                # Replace special characters that cannot be encoded in 'latin-1'
                text_content = word['content'].encode('ascii', 'ignore').decode('ascii')
            except Exception as e:
                print(f"Error processing text {word['content']}: {e}")
                text_content = "Error"
            draw.polygon(bbox, outline="red")
            draw.text((bbox[0][0], bbox[0][1]), text_content, fill="green")
    
    return annotated_image

page_width, page_height = letter 

# Function to calculate scale to fit the image within page dimensions
def calculate_scale(image, max_width, max_height):
    scale_w = max_width / image.width
    scale_h = max_height / image.height
    return min(scale_w, scale_h)


def get_text_density_map(pages):
    density_map = {}
    for page in pages:
        for line in page.lines:
            points = line.polygon
            if points:
                x_center = sum(point.x for point in points) / len(points)
                y_center = sum(point.y for point in points) / len(points)
                density_map[(x_center, y_center)] = density_map.get((x_center, y_center), 0) + 1
    return density_map

def find_highest_density_area(density_map):
    # This function will find the center of the area with the highest text density
    # For simplicity, this example just returns the center with the highest count
    # In a real scenario, you might want to consider a more sophisticated method
    # that takes into account the size and proximity of the high-density areas
    return max(density_map, key=density_map.get)

def crop_image_to_text(image_path, density_center, crop_size):
    with Image.open(image_path) as img:
        # Calculate the coordinates for the crop
        left = max(density_center[0] - crop_size[0] // 2, 0)
        upper = max(density_center[1] - crop_size[1] // 2, 0)
        right = min(density_center[0] + crop_size[0] // 2, img.width)
        lower = min(density_center[1] + crop_size[1] // 2, img.height)

        # Debug output
        print(f"Cropping coordinates: left={left}, upper={upper}, right={right}, lower={lower}")

        # Perform the crop
        cropped_img = img.crop((left, upper, right, lower))
        return cropped_img
    
    
def get_text_bounding_boxes(pages):
    bounding_boxes = []
    for page in pages:
        for line in page.lines:
            if line.polygon:
                box = [(point.x, point.y) for point in line.polygon]
                bounding_boxes.append(box)
    return bounding_boxes

def combine_text_regions(image_path, bounding_boxes):
    original_image = Image.open(image_path)

    # Calculate the combined bounding box
    min_x = min(min(box, key=lambda x: x[0])[0] for box in bounding_boxes)
    min_y = min(min(box, key=lambda x: x[1])[1] for box in bounding_boxes)
    max_x = max(max(box, key=lambda x: x[0])[0] for box in bounding_boxes)
    max_y = max(max(box, key=lambda x: x[1])[1] for box in bounding_boxes)

    # Create a new blank image with integer dimensions
    combined_image = Image.new('RGB', (int(max_x - min_x), int(max_y - min_y)), (255, 255, 255))
    
    for box in bounding_boxes:
        cropped_region = original_image.crop((min(box, key=lambda x: x[0])[0], 
                                              min(box, key=lambda x: x[1])[1], 
                                              max(box, key=lambda x: x[0])[0], 
                                              max(box, key=lambda x: x[1])[1]))
        # Paste the cropped region at integer coordinates
        combined_image.paste(cropped_region, (int(box[0][0] - min_x), int(box[0][1] - min_y)))

    return combined_image






###################################################
def parse_document_content(content):
    openai.api_key = 'your-api-key'

    try:
        response = openai.Completion.create(
            model="gpt-4",
            prompt=f"Extract specific information from the following text: {content}\n\nSpecies Name: ",
            max_tokens=100
            # Add additional parameters as needed
        )
        parsed_data = response.choices[0].text.strip()
        return parsed_data
    except Exception as e:
        print("An error occurred:", e)
        return None
####################################################

def analyze_text_density_and_crop(image_path):
    document_analysis_client = DocumentAnalysisClient(endpoint=endpoint, credential=AzureKeyCredential(key))
    
    with open(image_path, "rb") as f:
        image_stream = f.read()

    poller = document_analysis_client.begin_analyze_document("prebuilt-read", image_stream)
    result = poller.result()

    # Get bounding boxes of text regions
    bounding_boxes = get_text_bounding_boxes(result.pages)

    # Combine the text regions into one image
    combined_image = combine_text_regions(image_path, bounding_boxes)

    # Save the combined image temporarily and return its path
    combined_image_path = '/tmp/combined_image.png'
    combined_image.save(combined_image_path)
    return combined_image_path


def analyze_read(image_path, output_path, show_first_output=False):
    combined_image_path = analyze_text_density_and_crop(image_path)

    try:
        # Process the combined image with Azure Form Recognizer
        with open(combined_image_path, "rb") as f:
            combined_image_stream = f.read()

        document_analysis_client = DocumentAnalysisClient(
            endpoint=endpoint, credential=AzureKeyCredential(key)
        )

        poller = document_analysis_client.begin_analyze_document(
            "prebuilt-read", combined_image_stream)
        result = poller.result()

        # Collect words, their polygon data, and confidence
        words = []
        for page in result.pages:
            for word in page.words:
                words.append({
                    'content': word.content,
                    'polygon': word.polygon
                })

        # Prepare annotated image
        annotated_img = draw_boxes(combined_image_path, words)

        # Set up PDF
        output_filename = os.path.join(output_path, sanitize_filename(os.path.basename(image_path).replace('.jpg', '.pdf')))
        c = canvas.Canvas(output_filename, pagesize=letter)

        # Draw original image
        original_image = Image.open(image_path)
        scale = calculate_scale(original_image, page_width, page_height)
        img_width, img_height = original_image.width * scale, original_image.height * scale
        c.drawImage(image_path, 0, page_height - img_height, width=img_width, height=img_height, mask='auto')
        y_position = page_height - img_height

        # Draw annotated combined image
        annotated_image_path = '/tmp/annotated_image.png'
        annotated_img.save(annotated_image_path)
        scale = calculate_scale(annotated_img, page_width, page_height)
        annotated_img_width, annotated_img_height = annotated_img.width * scale, annotated_img.height * scale
        if y_position - annotated_img_height >= 0:
            c.drawImage(annotated_image_path, 0, y_position - annotated_img_height, width=annotated_img_width, height=annotated_img_height, mask='auto')
        else:
            c.showPage()  # Start a new page if not enough space
            c.drawImage(annotated_image_path, 0, page_height - annotated_img_height, width=annotated_img_width, height=annotated_img_height, mask='auto')

        # Add text
        textobject = c.beginText()
        textobject.setTextOrigin(10, y_position - 15)
        textobject.setFont("Times-Roman", 12)

        document_content = '\n'.join([word['content'] for word in words])
        for line in document_content.split('\n'):
            if textobject.getY() - 15 < 0:  # Check if new page is needed for more text
                c.drawText(textobject)
                c.showPage()
                textobject = c.beginText()
                textobject.setTextOrigin(10, page_height - 15)
                textobject.setFont("Times-Roman", 12)
            textobject.textLine(line)

        c.drawText(textobject)
        c.save()

    except Exception as e:
        print(f"An error occurred while processing {combined_image_path}: {e}")




if __name__ == "__main__":
    input_folder = '/projectnb/sparkgrp/ml-herbarium-grp/fall2023/resized-images'
    output_folder = '/projectnb/sparkgrp/ml-herbarium-grp/fall2023/AzureVision-results'

    # Create the output folder if it doesn't exist
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    image_counter = 0  # Initialize a counter for the number of images processed

    # Iterate over each image in the input folder
    for image_file in os.listdir(input_folder):
        image_path = os.path.join(input_folder, image_file)
        
        # Check if the file is an image
        if image_path.lower().endswith(('.png', '.jpg', '.jpeg')):
            analyze_read(image_path, output_folder, show_first_output=not first_output_shown)
            image_counter += 1  # Increment the counter

            if image_counter >= 1:  # Stop after processing 5 images
                break



